In [86]:
# hw3 convex optimization
# Import packages.
import cvxpy as cp
import cvxopt as cpt
import scipy.linalg
import numpy as np
import matplotlib.pyplot as pt
import time

from scipy.io import loadmat
from sklearn.metrics import accuracy_score

e=0.00005 #nesterov, values to be changed for analysis
np.random.seed(645632)
def svmplt(A_test, B_test, A, B, u, v, title='SVM'):
    pt.figure()
    # training 1
    pt.scatter(A[0,:],A[1,:],c='green')
    # training -1
    pt.scatter(B[0,:], B[1,:],c='blue')
    # markers for 1 and -1
    pt.scatter((A@u)[0],(A@u)[1],c='magenta', marker='.')
    pt.scatter((B@v)[0],(B@v)[1],c='cyan', marker='.')
    x = np.array(pt.axis()[0:2])
    md=0.5*(A@u+B@v)
    slope=(A@u-B@v)[1]/(A@u-B@v)[0]
    slope=-1/slope
    intercept=md[1]-slope*md[0]
    y=intercept+slope*x
    tl1=np.sign(-A_test[1,:]+intercept+(slope*A_test[0,:])).astype(int) #test label 1
    tl2=np.sign(-B_test[1,:]+intercept+(slope*B_test[0,:])).astype(int) #test label -1
    ytrue=[1]*A_test.shape[1]+[-1]*B_test.shape[1]
    ypredict=list(tl1)+list(tl2)
    print(title,"error",(1-accuracy_score(ytrue, ypredict))*100)
    pt.plot(x, y, c='red')
    pt.title(label=title+'_train')
    pt.savefig(title+'_train.png')
    pt.close()
    pt.figure()
    # test set label 1 in red
    pt.scatter((A@u)[0],(A@u)[1],c='magenta',marker='.') #magenta marker 1
    pt.scatter((B@v)[0],(B@v)[1],c='cyan',marker='.') #cyan marker -1
    pt.scatter(A_test[0,:], A_test[1,:], c='green') #green hyperplane
    # test set label -1 in blue
    pt.scatter(B_test[0,:],B_test[1,:],c='blue')
    pt.plot(x,y,c='red')
    pt.title(label=title+'_test')
    pt.savefig(title+'_test.png')
    pt.close()
    
# defining, using and plotting 1a
np.random.seed(500000) #for projected gradient
res=dict()
res['separable'] = dict()
res['overlap'] = dict()
train_separable = loadmat('svm data test/separable_case/train_separable.mat')
A=train_separable['A']
B=train_separable['B']
test_separable = loadmat('svm data test/separable_case/test_separable.mat')
X_test=test_separable['X_test'].T
labels=(test_separable['true_labels'].T)[:, 0]
#allocating the test data label 1 for A and -1 for B
A_test=X_test[np.where(labels == 1)].T
B_test=X_test[np.where(labels == -1)].T
m=A.shape[0] #rows
n=A.shape[1] #columns

# 1a) Define and solve the convex hull problem.
x=cp.Variable(n)
y=cp.Variable(n)
objfun=cp.sum_squares(A@x-B@y) #minimising the objective func
constraints=[x>=0,y>=0,cp.sum(x)==1,cp.sum(y)==1]
problem=cp.Problem(cp.Minimize(objfun), constraints)
s=time.time()
problem.solve()
totaltime=time.time()-s
res['separable']['P1'] = dict()
res['separable']['P1']['u'] = np.expand_dims(x.value, axis=1)
res['separable']['P1']['v'] = np.expand_dims(y.value, axis=1)
res['separable']['P1']['funcval'] = cp.sum_squares(A @ x - B @ y).value
res['separable']['P1']['iter'] = dict()
res['separable']['P1']['iter']['time'] = np.array([totaltime])
res['separable']['P1']['iter']['funcval'] = np.array([cp.sum_squares(A @ x - B @ y).value])
res['separable']['P1']['iter']['u'] = np.array([np.expand_dims(x.value, axis=1)])
res['separable']['P1']['iter']['v'] = np.array([np.expand_dims(y.value, axis=1)])
svmplt(A_test, B_test, A, B, np.expand_dims(x.value, axis=1),np.expand_dims(y.value, axis=1), title='svm data test/P1/cvx_hull')

#2a Projected gradient with 1a
"""
#reference:https://gist.github.com/mblondel/6f3b7aaad90606b98f71?permalink_comment_id=3977965
2a for 1a error is 0
"""
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u=np.sort(v)[::-1]
    cssv=np.cumsum(u) - z
    ind=np.arange(n_features) + 1
    cond=u - cssv / ind > 0
    rho=ind[cond][-1]
    theta=cssv[cond][-1] / float(rho)
    w=np.maximum(v - theta, 0)
    return w

u=np.random.random((n, 1))
v=np.random.random((n, 1))
alpha=0.0005
up=np.zeros_like(u)
vp=np.zeros_like(v)
up[:, 0]=projection_simplex_sort(u[:, 0])
vp[:, 0]=projection_simplex_sort(v[:, 0])
oldfuncval=np.power(np.linalg.norm(A@up-B@vp), 2)
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(5000):
    oldfuncval=np.power(np.linalg.norm(A@up-B@vp), 2)
    gu=2.0*A.T@A@up-2.0*A.T@B@vp
    gv=2.0*B.T@B@vp-2.0*B.T@A@up
    # gradient
    up=up-alpha*gu
    vp=vp-alpha*gv
    # projected gradient
    up[:, 0]=projection_simplex_sort(up[:, 0])
    vp[:, 0]=projection_simplex_sort(vp[:, 0])
    # the computation of the function value
    newfuncval=np.power(np.linalg.norm(A@up-B@vp), 2)
    ul.append(up)
    vl.append(vp)
    funclist.append(newfuncval)
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval) < e:
        break
svmplt(A_test, B_test, A, B, up, vp,title='svm data test/P1/PG')
res['separable']['PG'] = dict()
res['separable']['PG']['u'] = up
res['separable']['PG']['v'] = vp
res['separable']['PG']['funcval'] = newfuncval
res['separable']['PG']['iter'] = dict()
res['separable']['PG']['iter']['time'] = np.array(tl)
res['separable']['PG']['iter']['funcval'] = np.array(funclist)
res['separable']['PG']['iter']['u'] = np.array(ul)
res['separable']['PG']['iter']['v'] = np.array(vl)

#2b Nestrov accelerated gradient with 1a
"""
#reference: https://github.com/GRYE/Nesterov-accelerated-gradient-descent/blob/master/nesterov_method.py    
"""
e=0.00005 #nesterov
R=np.zeros((m, n*2)) #R mat of zeroes for resp dim
R[0:m,0:n]=A
R[0:m,n:]=-B
hess=2.0*R.T@R #computation of hessian
Lipschitz=scipy.linalg.svdvals(hess)[0] #singular value of matrix computation
x0=np.zeros((2*n,1))
x1=np.zeros((2*n,1))
x0[0:n,0],x0[n:,0] = projection_simplex_sort(x0[0:n, 0]), projection_simplex_sort(x0[n:, 0])
x1[0:n,0],x1[n:,0] = projection_simplex_sort(x1[0:n, 0]), projection_simplex_sort(x1[n:, 0])
y1=np.copy(x1)
a0=1
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(10000):
    oldfuncval = np.power(np.linalg.norm(R@x1),2)
    a1=(1+np.sqrt(4*a0*a0+1))/2
    tr=(a0-1)/a1
    y1=(1+tr)*x1 - tr*x0
    x0=x1
    a0=a1
    grad=2.0*R.T@R@y1
    x1=y1-(1/Lipschitz)*grad
    x1[0:n, 0], x1[n:, 0] = projection_simplex_sort(x1[0:n, 0]),projection_simplex_sort(x1[n:,0])
    newfuncval=np.power(np.linalg.norm(R@x1), 2)
    funclist.append(newfuncval)
    ul.append(x1[0:n])
    vl.append(x1[n:])
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval)<e and i>0:
        break
res['separable']['N']=dict()
res['separable']['N']['u']=x1[0:n]
res['separable']['N']['v']=x1[n:]
res['separable']['N']['funcval']=np.power(np.linalg.norm(R@x1),2)
res['separable']['N']['iter']=dict()
res['separable']['N']['iter']['time']=np.array(tl)
res['separable']['N']['iter']['funcval']=np.array(funclist)
res['separable']['N']['iter']['u']=np.array(ul)
res['separable']['N']['iter']['v']=np.array(vl)
svmplt(A_test,B_test,A,B,x1[0:n],x1[n:],title='svm data test/P1/nesterov')

#3a ADMM
"""
"""
#svm data test/P1a/ADMM error 0.0
def vp1(vec1, n):
    vec2 = np.zeros_like(vec1)
    vec2[0:n, 0], vec2[n:, 0] = projection_simplex_sort(vec1[0:n, 0]), projection_simplex_sort(vec1[n:, 0])
    return vec2

def vp2(vec1, n, d):
    vec2 = np.zeros_like(vec1)
    vec2 = np.clip(vec1, np.min(vec1), d)
    vec2[0:n, 0], vec2[n:, 0] = projection_simplex_sort(vec1[0:n, 0]), projection_simplex_sort(vec1[n:, 0])
    return vec2

e=0.05 #ADMM
m=A.shape[0] #getting rows and columns
n=A.shape[1]
R=np.zeros((m, n*2)) #creating a 0 mat of resp dim
R[0:m,0:n]=A
R[0:m,n:]=-B
x0=np.random.random((2*n, 1))
x1=vp1(x0, n)
rho=50
mu=x1-x0
matinv=np.linalg.inv(R.T@R+rho*np.eye(2*n))#compute the multiplicative inverse
a0=1
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(100000):
    oldfuncval = np.power(np.linalg.norm(R@x1),2)#2 norm square
    x0=matinv@(rho*x1-mu)
    x1=vp1(x0+(mu/rho),n)
    mu=mu+rho*(x0-x1)
    newfuncval = np.power(np.linalg.norm(R@x1),2)
    ul.append(x1[0:n])
    vl.append(x1[n:])
    funclist.append(newfuncval)
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval)<e and i > 1:
        break
res['separable']['ADMM']=dict()
res['separable']['ADMM']['u']=x1[0:n]
res['separable']['ADMM']['v']=x1[n:]
res['separable']['ADMM']['funcval']=np.power(np.linalg.norm(R@x1),2)
res['separable']['ADMM']['iter']=dict()
res['separable']['ADMM']['iter']['time']=np.array(tl)
res['separable']['ADMM']['iter']['funcval']=np.array(funclist)
res['separable']['ADMM']['iter']['u']=np.array(ul)
res['separable']['ADMM']['iter']['v']=np.array(vl)
svmplt(A_test,B_test,A,B,x1[0:n],x1[n:],title='svm data test/P1/ADMM')



svm data test/P1/cvx_hull error 0.0
svm data test/P1/PG error 0.0
svm data test/P1/nesterov error 0.0
svm data test/P1/ADMM error 0.0


1b: Applying all algorithms: PG, Nesterov, ADMM

In [89]:
##############################################################################
##############################################################################
##############################################################################

# 1b) Define and solve the reduced convex hull problem
# 1b) error is :svm data test/P1b/red_cvx_hull error 2.0000000000000018 d=0.06, high misclassification
#d=0.05, 0.045 svm data test/P1b/red_cvx_hull error 1.5000000000000013, less amount of misclassification
#d= 0.03 svm data test/P1b/red_cvx_hull error 2.0000000000000018
train_overlap=loadmat('svm data test/overlap_case/train_overlap.mat')
A=train_overlap['A'] #reading the 'A' from train overlap dataset to A
B=train_overlap['B'] # reading the 'B' from train overlap into B
test_overlap=loadmat('svm data test/overlap_case/test_overlap.mat')
X_test = test_overlap['X_test'].T
labels = (test_overlap['true_labels'].T)[:, 0]
A_test = X_test[np.where(labels == 1)].T
B_test = X_test[np.where(labels == -1)].T
d=0.05 #appropriate d for drawing the hyperplane that bisects the samples.
m=A.shape[0]
n=A.shape[1]
# Define and solve the reduced cvx hull problem.
x=cp.Variable(n)
y=cp.Variable(n)
objfun=cp.sum_squares(A@x-B@y)
constraints=[x>=0,y>=0,x<=d,y<=d,cp.sum(x)==1,cp.sum(y)==1]
problem=cp.Problem(cp.Minimize(objfun), constraints)
s=time.time()
problem.solve()
totaltime=time.time()-s
res['overlap']['P1']=dict()
res['overlap']['P1']['u']=np.expand_dims(x.value,axis=1)
res['overlap']['P1']['v']=np.expand_dims(y.value,axis=1)
res['overlap']['P1']['funcval']=cp.sum_squares(A@x-B@y).value
res['overlap']['P1']['iter']=dict()
res['overlap']['P1']['iter']['time']=np.array([totaltime])
res['overlap']['P1']['iter']['funcval']=np.array([cp.sum_squares(A@x-B@y).value])
res['overlap']['P1']['iter']['u']=np.array([np.expand_dims(x.value, axis=1)])
res['overlap']['P1']['iter']['v']=np.array([np.expand_dims(y.value, axis=1)])
svmplt(A_test, B_test, A, B, np.expand_dims(x.value, axis=1),np.expand_dims(y.value, axis=1), title='svm data test/P1/red_cvx_hull')

#2a Projected gradient with 1b
u=np.random.random((n, 1))
v=np.random.random((n, 1))
alpha=0.005
up=np.zeros_like(u)
vp=np.zeros_like(v)
up[:, 0]=projection_simplex_sort(u[:, 0])
vp[:, 0]=projection_simplex_sort(v[:, 0])
oldfuncval=np.power(np.linalg.norm(A@up-B@vp), 2)
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(5000):
    oldfuncval = np.power(np.linalg.norm(A@up-B@vp), 2)
    gu = 2.0*A.T@A@up-2.0*A.T@B@vp
    gv = 2.0*B.T@B@vp-2.0*B.T@A@up
    # gradient
    up = up-alpha*gu
    vp = vp-alpha*gv
    # projected gradient
    up[:, 0] = projection_simplex_sort(up[:, 0])
    vp[:, 0] = projection_simplex_sort(vp[:, 0])
    # the computation of the function value
    newfuncval = np.power(np.linalg.norm(A@up-B@vp), 2)
    ul.append(up)
    vl.append(vp)
    funclist.append(newfuncval)
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval) < e:
        break
svmplt(A_test, B_test, A, B, up, vp,title='svm data test/P1/PG_1b')
res['overlap']['PG'] = dict()
res['overlap']['PG']['u'] = up
res['overlap']['PG']['v'] = vp
res['overlap']['PG']['funcval'] = newfuncval
res['overlap']['PG']['iter'] = dict()
res['overlap']['PG']['iter']['time'] = np.array(tl)
res['overlap']['PG']['iter']['funcval'] = np.array(funclist)
res['overlap']['PG']['iter']['u'] = np.array(ul)
res['overlap']['PG']['iter']['v'] = np.array(vl)


#2b Nestrov acceleration with 1b
"""
Algorithm for Nestrov Acceleration:

"""
e=0.045 #nesterov
R=np.zeros((m, n*2)) #R mat of zeroes for resp dim
R[0:m,0:n]=A
R[0:m,n:]=-B
hess=2.0*R.T@R #computation of hessian
Lipschitz=scipy.linalg.svdvals(hess)[0] #singular value of matrix computation
x0=np.zeros((2*n,1))
x1=np.zeros((2*n,1))
x0[0:n,0],x0[n:,0] = projection_simplex_sort(x0[0:n, 0]), projection_simplex_sort(x0[n:, 0])
x1[0:n,0],x1[n:,0] = projection_simplex_sort(x1[0:n, 0]), projection_simplex_sort(x1[n:, 0])
y1=np.copy(x1)
a0=1
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(10000):
    oldfuncval = np.power(np.linalg.norm(R@x1),2)
    a1=(1+np.sqrt(4*a0*a0+1))/2
    tr=(a0-1)/a1
    y1=(1+tr)*x1 - tr*x0
    x0=x1
    a0=a1
    grad=2.0*R.T@R@y1
    x1=y1-(1/Lipschitz)*grad
    x1[0:n, 0], x1[n:, 0] = projection_simplex_sort(x1[0:n, 0]),projection_simplex_sort(x1[n:,0])
    newfuncval=np.power(np.linalg.norm(R@x1), 2)
    funclist.append(newfuncval)
    ul.append(x1[0:n])
    vl.append(x1[n:])
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval)<e and i>0:
        break
res['overlap']['N']=dict()
res['overlap']['N']['u']=x1[0:n]
res['overlap']['N']['v']=x1[n:]
res['overlap']['N']['funcval']=np.power(np.linalg.norm(R@x1),2)
res['overlap']['N']['iter']=dict()
res['overlap']['N']['iter']['time']=np.array(tl)
res['overlap']['N']['iter']['funcval']=np.array(funclist)
res['overlap']['N']['iter']['u']=np.array(ul)
res['overlap']['N']['iter']['v']=np.array(vl)
svmplt(A_test,B_test,A,B,x1[0:n],x1[n:],title='svm data test/P1/nesterov_1b')

#3 for 1b ADMM
"""
ADMM for 1b
svm data test/P1b/ADMM error 80.0 for e=0.05, iteration= 1000 to 10000
svm data test/P1b/ADMM error 1.5000000000000013 for e= 0.05, iteration  1000 to 10000
heavily depends on value of e, for error
"""
e=0.005 #ADMM
#e=0.05
m=A.shape[0] #getting rows and columns
n=A.shape[1]
R=np.zeros((m,n*2)) #creating a 0 mat of resp dim
R[0:m,0:n]=A
R[0:m,n:]=-B
x0=np.random.random((2*n, 1))
x1=vp1(x0, n)
rho=50
mu=x1-x0
matinv=np.linalg.inv(R.T@R+rho*np.eye(2*n))#compute the multiplicative inverse
a0=1
tl=list()
ul=list()
vl=list()
funclist=list()
s=time.time()
for i in range(1000):
    oldfuncval = np.power(np.linalg.norm(R@x1),2)#2 norm square
    x0=matinv@(rho*x1-mu)
    x1=vp1(x0+(mu/rho),n)
    mu=mu+rho*(x0-x1)
    newfuncval = np.power(np.linalg.norm(R@x1),2)
    ul.append(x1[0:n])
    vl.append(x1[n:])
    funclist.append(newfuncval)
    tl.append(time.time()-s)
    if np.abs(oldfuncval-newfuncval)<e and i > 1:
        break
res['overlap']['ADMM']=dict()
res['overlap']['ADMM']['u']=x1[0:n]
res['overlap']['ADMM']['v']=x1[n:]
res['overlap']['ADMM']['funcval']=np.power(np.linalg.norm(R@x1),2)
res['overlap']['ADMM']['iter']=dict()
res['overlap']['ADMM']['iter']['time']=np.array(tl)
res['overlap']['ADMM']['iter']['funcval']=np.array(funclist)
res['overlap']['ADMM']['iter']['u']=np.array(ul)
res['overlap']['ADMM']['iter']['v']=np.array(vl)
svmplt(A_test,B_test,A,B,x1[0:n],x1[n:],title='svm data test/P1/ADMM_1b')




svm data test/P1/red_cvx_hull error 1.5000000000000013
svm data test/P1/PG_1b error 2.0000000000000018
svm data test/P1/nesterov_1b error 1.5000000000000013
svm data test/P1/ADMM_1b error 1.5000000000000013


Problem 4 for 1a and 1b

In [88]:
colorforplt={'P1': 'green','PG': 'red','N': 'black','ADMM': 'blue'}
for key in res.keys():
    for exp in res[key]:
        #print(exp)
        if exp!='P1':
            pt.title(label=key+'_'+exp+'_iteration')
            pt.xlabel('iteration steps')
            pt.ylabel('function value')
            pt.plot(np.arange(res[key][exp]['iter']['funcval'].shape[0]),res[key][exp]['iter']['funcval'],c=colorforplt[exp])
            pt.savefig('svm data test/plots/'+key+'_'+exp+'_iteration.png')
            pt.close()
            
    for exp in res[key]:
        if exp!='P1':
            pt.title(label=key+'_'+exp+'_time')
            pt.xlabel('time (secs)')
            pt.ylabel(exp+' function value')
            pt.plot(res[key][exp]['iter']['time'], res[key][exp]['iter']['funcval'],c=colorforplt[exp])
            pt.savefig('svm data test/plots/'+key+'_'+exp+'_time.png')
            pt.close()
    #time comparison
    pt.title("Runtime Comparison Plot "+key)
    pt.xlabel('For P1,PG,Nesterov,ADMM')
    pt.ylabel('time taken')
    pt.scatter([1,2,3,4],[res[key]['P1']['iter']['time'][-1],res[key]['PG']['iter']['time'][-1],res[key]['N']['iter']['time'][-1],res[key]['ADMM']['iter']['time'][-1]])
    pt.savefig('svm data test/comparison_plots/'+key+'_timecompare.png')
    pt.close()  
    #iteration comparison
    pt.title("Iteration Comparison Plot "+key)
    pt.xlabel('P1,PG,Nesterov,ADMM')
    pt.ylabel('number of iterations')
    pt.scatter([1,2,3,4],[len(res[key]['P1']['iter']['time']),len(res[key]['PG']['iter']['time']),len(res[key]['N']['iter']['time']),len(res[key]['ADMM']['iter']['time'])])
    pt.savefig('svm data test/comparison_plots/'+key+'_itercompare.png')       
    print("The", key, " time taken: P1->", res[key]['P1']['iter']['time'][-1], " PG ", res[key]['PG']
          ['iter']['time'][-1], " Nesterov", res[key]['N']['iter']['time'][-1], " ADMM ", res[key]['ADMM']['iter']['time'][-1])
    print("The", key, " optimal function value: P1->", res[key]['P1']['funcval'], " PG ", res[key]['PG']
          ['funcval'], " nesterov", res[key]['N']['funcval'], " admm", res[key]['ADMM']['funcval'])
    pt.close() 

The separable  time taken: P1-> 0.017314910888671875  PG  0.03226208686828613  Nesterov 0.0200650691986084  ADMM  0.008076190948486328
The separable  optimal function value: P1-> 80.01935301468488  PG  80.01935301473766  nesterov 80.01935301473766  admm 80.01935301473766
The overlap  time taken: P1-> 0.04652595520019531  PG  0.0011780261993408203  Nesterov 0.0014488697052001953  ADMM  0.0010731220245361328
The overlap  optimal function value: P1-> 0.00035136463974322775  PG  0.035855982586354446  nesterov 0.07142501536531877  admm 0.016545888413175273


P1
PG
N
ADMM
P1
PG
N
ADMM
